Learning Objectives:
 1. Use Natural Language Query DBs to interact with relational DBs

In [2]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
import os
import numexpr as ne
os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'


In [4]:
os.environ["OPENAI_API_KEY"] = "<Your API Key"

import openai
openai.api_key = "<Your API Key"

In [6]:
from sqlalchemy import create_engine, Column, Table, String, MetaData


#Create a SQL table
metadata_obj = MetaData()
engine = create_engine('sqlite:///databases/cats.db', future=True)

cat_breeds_table = Table(
    "cat_breeds",
    metadata_obj,
    Column("name", String(16), primary_key=True),
    Column("origin", String(16), nullable=False),
    Column("temperament", String(16), nullable=False),
    Column("size", String(16), nullable=False),
    Column("social_behavior", String(16), nullable=False),
)

metadata_obj.create_all(engine)

In [7]:
from sqlalchemy import insert

breeds_data = [
    {
        'name': 'Savannah',
        'origin': 'USA',
        'temperament': 'high energy',
        'size': 'Large',
        'social_behavior': 'sociable'
    },
    {
        'name': 'Ragdoll',
        'origin': 'USA',
        'temperament': 'gentle',
        'size': 'Large',
        'social_behavior': 'sociable'
    },
    {
        'name': 'Maine Coon',
        'origin': 'USA (Maine)',
        'temperament': 'friendly',
        'size': 'Very Large',
        'social_behavior': 'sociable'
    }
]
for breed in breeds_data:
    stmt = insert(cat_breeds_table).values(**breed)
    with engine.connect() as connection:
        cursor = connection.execute(stmt)
        connection.commit()

In [8]:
with engine.connect() as connection:
    result = connection.exec_driver_sql("SELECT * FROM cat_breeds")
    print(result.fetchall())

[('Savannah', 'USA', 'high energy', 'Large', 'sociable'), ('Ragdoll', 'USA', 'gentle', 'Large', 'sociable'), ('Maine Coon', 'USA (Maine)', 'friendly', 'Very Large', 'sociable')]


In [9]:
from llama_index import SQLDatabase
sql_database = SQLDatabase(engine, include_tables=["cat_breeds"])

In [10]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
llm = OpenAI(model="gpt-4", temperature=0, verbose=True)
service_context = ServiceContext.from_defaults(llm=llm)


In [11]:
from llama_index.indices.struct_store import NLSQLTableQueryEngine

# creates a natural language SQL table query engine, specifying the SQL database and 
# additional context information for querying the "cat_breeds" table.
query_engine = NLSQLTableQueryEngine(
    sql_database,
    context_query_kwargs={"cat_breeds": (
        "The only columns available are: name,origin,temperament,size,social_behavior. Do not use other columns and foreign keys. \n"
        "Do not attempt to run a query if the column is not among available columns.\n"
        "Do not use unexistant colums in a WHERE part of the query. \n"
    )}
)

In [12]:
openai.log = "debug"

In [13]:
# a query is executed using the NLSQLTableQueryEngine, and the response is stored in the variable response.
response = query_engine.query("What kind of a temperament does a Ragdoll cat have?")

INFO:llama_index.indices.struct_store.sql_retriever:> Table desc str: Table 'cat_breeds' has columns: name (VARCHAR(16)), origin (VARCHAR(16)), temperament (VARCHAR(16)), size (VARCHAR(16)), social_behavior (VARCHAR(16)), and foreign keys: . The table description is: The only columns available are: name,origin,temperament,size,social_behavior. Do not use other columns and foreign keys. 
Do not attempt to run a query if the column is not among available columns.
Do not use unexistant colums in a WHERE part of the query. 

> Table desc str: Table 'cat_breeds' has columns: name (VARCHAR(16)), origin (VARCHAR(16)), temperament (VARCHAR(16)), size (VARCHAR(16)), social_behavior (VARCHAR(16)), and foreign keys: . The table description is: The only columns available are: name,origin,temperament,size,social_behavior. Do not use other columns and foreign keys. 
Do not attempt to run a query if the column is not among available columns.
Do not use unexistant colums in a WHERE part of the query. 

In [14]:
print(response)

A Ragdoll cat is known for having a gentle temperament.


In [15]:
response.metadata

{'7e7fa347-9c15-4839-9b77-d179608195b9': {},
 'sql_query': "SELECT temperament\nFROM cat_breeds\nWHERE name = 'Ragdoll';",
 'result': [('gentle',)],
 'col_keys': ['temperament']}